In [1]:
!pip install sqlalchemy mysql-connector-python

In [2]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json"

headers = {
    "accept": "application/json",
    "x-api-key": "1k3d2rnV453wSlaFM5SHAxf7PwoYTNj9CS6SqukZ"
}

response = requests.get(url, headers=headers)
print(response.text)

{"generated_at":"2025-06-24T10:13:13+00:00","competitions":[{"id":"sr:competition:620","name":"Hopman Cup","type":"mixed","gender":"mixed","category":{"id":"sr:category:181","name":"Hopman Cup"}},{"id":"sr:competition:660","name":"World Team Cup","type":"mixed","gender":"men","category":{"id":"sr:category:3","name":"ATP"},"level":"atp_250"},{"id":"sr:competition:990","name":"ATP Challenger Tour Finals","parent_id":"sr:competition:6239","type":"singles","gender":"men","category":{"id":"sr:category:72","name":"Challenger"}},{"id":"sr:competition:1207","name":"Championship International Series","type":"singles","gender":"women","category":{"id":"sr:category:6","name":"WTA"},"level":"wta_championships"},{"id":"sr:competition:2100","name":"Davis Cup","type":"mixed","gender":"men","category":{"id":"sr:category:76","name":"Davis Cup"}},{"id":"sr:competition:2102","name":"Billie Jean King Cup","type":"mixed","gender":"women","category":{"id":"sr:category:74","name":"Billie Jean King Cup"}},{"i

In [3]:
import mysql.connector

# Connect to MySQL (make sure your DB already exists)
conn = mysql.connector.connect(
    host="localhost",
    user="root",        
    password="Melmir@123",    
    database="david"     
)
cursor = conn.cursor()

# Create Categories Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS categories (
    category_id VARCHAR(50) PRIMARY KEY,
    category_name VARCHAR(100) NOT NULL
);
""")

# Create Competitions Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS competitions (
    competition_id VARCHAR(50) PRIMARY KEY,
    competition_name VARCHAR(100) NOT NULL,
    parent_id VARCHAR(50),
    type VARCHAR(20) NOT NULL,
    gender VARCHAR(10) NOT NULL,
    category_id VARCHAR(50),
    FOREIGN KEY (category_id) REFERENCES categories(category_id)
);
""")

conn.commit()
print("Tables created successfully.")

Tables created successfully.


In [4]:
import requests
import mysql.connector

API_KEY = '1k3d2rnV453wSlaFM5SHAxf7PwoYTNj9CS6SqukZ'
url = f"https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key={'1k3d2rnV453wSlaFM5SHAxf7PwoYTNj9CS6SqukZ'}"

# Step 1: Fetch data from API
response = requests.get(url)
data = response.json()

# Step 2: Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",        
    password="Melmir@123",    
    database="david" 
)
cursor = conn.cursor()

# Step 3: Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS categories (
    category_id VARCHAR(50) PRIMARY KEY,
    category_name VARCHAR(100) NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS competitions (
    competition_id VARCHAR(50) PRIMARY KEY,
    competition_name VARCHAR(100) NOT NULL,
    parent_id VARCHAR(50),
    type VARCHAR(20) NOT NULL,
    gender VARCHAR(10) NOT NULL,
    category_id VARCHAR(50),
    FOREIGN KEY (category_id) REFERENCES categories(category_id)
);
""")

# Step 4: Insert data into tables
for item in data.get("competitions", []):
    category = item["category"]
    category_id = category["id"]
    category_name = category["name"]

    # Insert into categories table (use IGNORE or ON DUPLICATE KEY to avoid duplicates)
    cursor.execute("""
        INSERT IGNORE INTO categories (category_id, category_name)
        VALUES (%s, %s)
    """, (category_id, category_name))

    # Insert into competitions table
    cursor.execute("""
        INSERT IGNORE INTO competitions (
            competition_id, competition_name, parent_id, type, gender, category_id
        )
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        item.get("id"),
        item.get("name"),
        item.get("parent_id"),
        item.get("type"),
        item.get("gender"),
        category_id
    ))

conn.commit()
print("Data inserted successfully from Sportradar API.")

# Step 5: Close connection
cursor.close()
conn.close()

Data inserted successfully from Sportradar API.


In [5]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",        
    password="Melmir@123",    
    database="david" 
)
cursor = conn.cursor()

# List all tables in the connected database
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
print("Tables in database:")
for (table_name,) in tables:
    print(f"- {table_name}")

Tables in database:
- categories
- competitions
- competitor_rankings
- competitors
- complexes
- marks
- students
- venues


In [6]:
#  List all competitions with their category name
cursor.execute("""
SELECT c.competition_id, c.competition_name, cat.category_name
FROM competitions c
JOIN categories cat ON c.category_id = cat.category_id;
""")
print("1. Competitions with Category:")
for row in cursor.fetchall():
    print(row)

1. Competitions with Category:
('sr:competition:12229', 'IPTL', 'IPTL')
('sr:competition:14814', 'Juniors AO, Melbourne, Australia Men Singles', 'Juniors')
('sr:competition:14816', 'Juniors AO, Melbourne, Australia Men Doubles', 'Juniors')
('sr:competition:14818', 'Juniors AO, Melbourne, Australia Women Singles', 'Juniors')
('sr:competition:14820', 'Juniors AO, Melbourne, Australia Women Doubles', 'Juniors')
('sr:competition:15399', 'Juniors French Open, Paris, France Men Singles', 'Juniors')
('sr:competition:15401', 'Juniors French Open, Paris, France Men Doubles', 'Juniors')
('sr:competition:15403', 'Juniors French Open, Paris, France Women Singles', 'Juniors')
('sr:competition:15405', 'Juniors French Open, Paris, France Women Doubles', 'Juniors')
('sr:competition:15660', 'Juniors Wimbledon, London, GB Women Singles', 'Juniors')
('sr:competition:15678', 'Juniors Wimbledon, London, GB Men Singles', 'Juniors')
('sr:competition:15680', 'Juniors Wimbledon, London, GB Men Doubles', 'Junio

In [7]:
# Count the number of competitions in each category
cursor.execute("""
SELECT cat.category_name, COUNT(c.competition_id)
FROM competitions c
JOIN categories cat ON c.category_id = cat.category_id
GROUP BY cat.category_name;
""")
print("\n2. Competition Count by Category:")
for row in cursor.fetchall():
    print(row)


2. Competition Count by Category:
('IPTL', 1)
('Juniors', 16)
('Legends', 11)
('Wheelchairs', 16)
('Hopman Cup', 1)
('ITF Women', 2032)
('Wheelchairs Juniors', 6)
('United Cup', 1)
('UTR Men', 120)
('UTR Women', 104)
('ATP', 223)
('WTA', 255)
('Challenger', 912)
('Billie Jean King Cup', 1)
('Davis Cup', 1)
('ITF Men', 2198)
('Exhibition', 32)
('WTA 125K', 180)


In [8]:
# Find all competitions of type 'doubles'
cursor.execute("""
SELECT competition_id, competition_name, type
FROM competitions
WHERE type = 'doubles';
""")
print("\n3. Doubles Competitions:")
for row in cursor.fetchall():
    print(row)


3. Doubles Competitions:
('sr:competition:10027', 'ITF Men Stara Zagora, Bulgaria Men Doubles', 'doubles')
('sr:competition:10033', 'ITF Men Sibiu, Romania Men Doubles', 'doubles')
('sr:competition:10039', 'ITF Men Busto Arsizio, Italy Men Doubles', 'doubles')
('sr:competition:10045', 'ITF Men Sabac, Serbia Men Doubles', 'doubles')
('sr:competition:10057', 'ITF Men Seefeld, Austria Men Doubles', 'doubles')
('sr:competition:10075', 'ATP Challenger Portoroz, Slovenia Men DoublesUTR Wo', 'doubles')
('sr:competition:10093', 'ITF Men Focsani, Romania Men Doubles', 'doubles')
('sr:competition:10153', 'ATP Challenger Eskisehir, Turkey Men Doubles', 'doubles')
('sr:competition:10159', 'ITF Men Tallinn, Estonia Men Doubles', 'doubles')
('sr:competition:10183', 'ITF Women Campos Do Jordao, Brazil Women Doubles', 'doubles')
('sr:competition:10225', 'ITF Men Misiones, Argentina Men Doubles', 'doubles')
('sr:competition:10255', 'ATP Challenger Liberec, Czech Republic Men Doubles', 'doubles')
('sr:

In [9]:
# Get competitions that belong to category 'ITF Men'
cursor.execute("""
SELECT c.competition_id, c.competition_name
FROM competitions c
JOIN categories cat ON c.category_id = cat.category_id
WHERE cat.category_name = 'ITF Men';
""")
print("\n4. Competitions in 'ITF Men' Category:")
for row in cursor.fetchall():
    print(row)


4. Competitions in 'ITF Men' Category:
('sr:competition:10025', 'ITF Men Stara Zagora, Bulgaria Men Singles')
('sr:competition:10027', 'ITF Men Stara Zagora, Bulgaria Men Doubles')
('sr:competition:10031', 'ITF Men Sibiu, Romania Men Singles')
('sr:competition:10033', 'ITF Men Sibiu, Romania Men Doubles')
('sr:competition:10037', 'ITF Men Busto Arsizio, Italy Men Singles')
('sr:competition:10039', 'ITF Men Busto Arsizio, Italy Men Doubles')
('sr:competition:10043', 'ITF Men Sabac, Serbia Men Singles')
('sr:competition:10045', 'ITF Men Sabac, Serbia Men Doubles')
('sr:competition:10055', 'ITF Men Seefeld, Austria Men Singles')
('sr:competition:10057', 'ITF Men Seefeld, Austria Men Doubles')
('sr:competition:10091', 'ITF Men Focsani, Romania Men Singles')
('sr:competition:10093', 'ITF Men Focsani, Romania Men Doubles')
('sr:competition:10157', 'ITF Men Tallinn, Estonia Men Singles')
('sr:competition:10159', 'ITF Men Tallinn, Estonia Men Doubles')
('sr:competition:10223', 'ITF Men Mision

In [10]:
# Identify parent competitions and their sub-competitions
cursor.execute("""
SELECT parent.competition_name AS parent_competition, child.competition_name AS sub_competition
FROM competitions child
JOIN competitions parent ON child.parent_id = parent.competition_id;
""")
print("\n5. Parent-Child Competition Relationships:")
for row in cursor.fetchall():
    print(row)


5. Parent-Child Competition Relationships:
('ITF Romania F9, Men Singles', 'ITF Romania F9, Men Doubles')


In [11]:
# Distribution of competition types by category
cursor.execute("""
SELECT cat.category_name, c.type, COUNT(*)
FROM competitions c
JOIN categories cat ON c.category_id = cat.category_id
GROUP BY cat.category_name, c.type;
""")
print("\n6. Distribution of Types by Category:")
for row in cursor.fetchall():
    print(row)


6. Distribution of Types by Category:
('IPTL', 'singles', 1)
('Juniors', 'singles', 8)
('Juniors', 'doubles', 8)
('Legends', 'doubles', 8)
('Legends', 'mixed_doubles', 3)
('Wheelchairs', 'singles', 8)
('Wheelchairs', 'doubles', 8)
('Hopman Cup', 'mixed', 1)
('ITF Women', 'singles', 1014)
('ITF Women', 'doubles', 1018)
('Wheelchairs Juniors', 'singles', 4)
('Wheelchairs Juniors', 'doubles', 2)
('United Cup', 'mixed', 1)
('UTR Men', 'singles', 120)
('UTR Women', 'singles', 104)
('ATP', 'singles', 108)
('ATP', 'doubles', 107)
('ATP', 'mixed_doubles', 6)
('ATP', 'mixed', 2)
('WTA', 'singles', 128)
('WTA', 'doubles', 127)
('Challenger', 'singles', 457)
('Challenger', 'doubles', 455)
('Billie Jean King Cup', 'mixed', 1)
('Davis Cup', 'mixed', 1)
('ITF Men', 'singles', 1099)
('ITF Men', 'doubles', 1099)
('Exhibition', 'singles', 22)
('Exhibition', 'doubles', 8)
('Exhibition', 'mixed_doubles', 2)
('WTA 125K', 'singles', 90)
('WTA 125K', 'doubles', 90)


In [12]:
# Competitions with no parent (Top-level competitions)
cursor.execute("""
SELECT competition_id, competition_name
FROM competitions
WHERE parent_id IS NULL;
""")
print("\n7. Top-Level Competitions (No Parent):")
for row in cursor.fetchall():
    print(row)


7. Top-Level Competitions (No Parent):
('sr:competition:1207', 'Championship International Series')
('sr:competition:12229', 'IPTL')
('sr:competition:15393', 'ITF Men San Jose, Costa Rica Men Singles')
('sr:competition:15395', 'ITF Men San Jose, Costa Rica Men Doubles')
('sr:competition:2100', 'Davis Cup')
('sr:competition:2102', 'Billie Jean King Cup')
('sr:competition:21028', 'ITF Colombia 02A, Women Singles')
('sr:competition:21228', 'ITF Australia 02B, Women Doubles')
('sr:competition:22334', 'ITF Uzbekistan 01A, Women Singles')
('sr:competition:22763', 'ITF Tunisia 39A, Women Doubles')
('sr:competition:22787', 'ITF Egypt F38, Men Doubles')
('sr:competition:22959', 'ITF USA 17A, Women Singles')
('sr:competition:22997', 'ITF Bosnia & Herzegovina 01A, Women Doubles')
('sr:competition:29204', 'ITF Mexico F13, Men Singles')
('sr:competition:29206', 'ITF Mexico F13, Men Doubles')
('sr:competition:29386', 'ITF Mexico 17A, Women Singles')
('sr:competition:31137', 'ITF Mexico F30, Men Sin

In [13]:
# COLLECT THE COMPLEXES DATA FROM THE API ENDPOINTS

In [14]:
import mysql.connector

# Connect to your database
conn = mysql.connector.connect(
    host="localhost",
    user="root",        
    password="Melmir@123",    
    database="david" 
)
cursor = conn.cursor()

# Create Complexes Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS complexes (
    complex_id VARCHAR(50) PRIMARY KEY,
    complex_name VARCHAR(100) NOT NULL
);
""")

# Create Venues Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS venues (
    venue_id VARCHAR(50) PRIMARY KEY,
    venue_name VARCHAR(100) NOT NULL,
    city_name VARCHAR(100) NOT NULL,
    country_name VARCHAR(100) NOT NULL,
    country_code CHAR(3) NOT NULL,
    timezone VARCHAR(100) NOT NULL,
    complex_id VARCHAR(50),
    FOREIGN KEY (complex_id) REFERENCES complexes(complex_id)
);
""")

conn.commit()
print("Complexes and Venues tables created.")

Complexes and Venues tables created.


In [15]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json"
headers = {
    "accept": "application/json",
    "x-api-key": "1k3d2rnV453wSlaFM5SHAxf7PwoYTNj9CS6SqukZ"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    for complex_ in data.get("complexes", []):
        complex_id = complex_["id"]
        complex_name = complex_["name"]

        # Insert into complexes
        cursor.execute("""
            INSERT IGNORE INTO complexes (complex_id, complex_name)
            VALUES (%s, %s)
        """, (complex_id, complex_name))

        # Insert each venue in that complex
        for venue in complex_.get("venues", []):
            cursor.execute("""
                INSERT IGNORE INTO venues (
                    venue_id, venue_name, city_name, country_name,
                    country_code, timezone, complex_id
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (
                venue.get("id"),
                venue.get("name"),
                venue.get("city_name"),
                venue.get("country_name"),
                venue.get("country_code"),
                venue.get("timezone"),
                complex_id
            ))

    conn.commit()
    print("Complex and venue data inserted successfully.")
else:
    print(f" Failed to fetch data. Status: {response.status_code}")

Complex and venue data inserted successfully.


In [16]:
#  List all venues with their complex name
cursor.execute("""
SELECT v.venue_name, c.complex_name
FROM venues v
JOIN complexes c ON v.complex_id = c.complex_id;
""")
print("\n1. Venues with Complex Name:")
for row in cursor.fetchall():
    print(row)


1. Venues with Complex Name:
('Estadio Monumental', 'Club Tennis Las Terrazas de Miraflores')
('Stadium', 'Shenzhen International Tennis Center')
('Court 2', 'Shenzhen Longgang Sports Center')
('Court 7', 'Olympic Park Tennis Center')
('Court 20', 'Melbourne Park')
('Court 22', 'Melbourne Park')
('Estadio Ivan Elias Moreno', 'Club Tennis Las Terrazas de Miraflores')
('Stade Numa-Daly Magenta', 'Complexe Tennistique Marie-Louise Lhuillier')
('Noevir Stadium', 'Miki Disaster Management Park')
('Shanghai Stadium', 'Qi Zhong Tennis Center')
('Cancha 5', 'Acapulco Princess Mundo Imperial')
('Court 13', 'Indian Wells Tennis Garden')
('Center Court', 'Royal Tennis Club de Marrakech')
('Court 1', 'Royal Tennis Club de Marrakech')
('Court 2', 'Royal Tennis Club de Marrakech')
('Court 4', 'Royal Tennis Club de Marrakech')
('SZ Court', 'MTTC Iphitos e.V.')
('Court 1', 'Complexo Desportivo do Jamor')
('Center Court', 'Tennis Club 1.FCN')
('Show Court', 'Tennis Club 1.FCN')
('Court 1', 'Tennis Clu

In [17]:
# Count the number of venues in each complex
cursor.execute("""
SELECT c.complex_name, COUNT(v.venue_id)
FROM venues v
JOIN complexes c ON v.complex_id = c.complex_id
GROUP BY c.complex_name;
""")
print("\n2. Venue Count per Complex:")
for row in cursor.fetchall():
    print(row)


2. Venue Count per Complex:
('Kindarena', 4)
('Estadio de la Cartuja', 2)
('Estadio German Becker', 4)
('Palais des sports de Gerland', 7)
('Sibur Arena', 4)
('Palais Des Sport', 7)
('Palasport', 4)
('Salle Steredenn', 6)
('Garanti Koza Arena', 1)
('Porsche-Arena', 3)
('Rotterdam Ahoy', 3)
('The O2', 1)
('Bluesun Tennis Center', 8)
('Club Sonoma, Estadio Gnp Seguros', 3)
('Accor Arena', 2)
('Stadium Goran Ivanisevic in ITC Stella Maris', 1)
('Roy Emerson Arena', 4)
('Complexo de Tenis do Jamor', 5)
('Club de Tenis Puente Romano', 9)
('National Tennis Center', 63)
('Canberra Tennis Centre', 7)
('BT Arena', 2)
('Valletta Cambiaso ASD', 9)
('Porsche Arena', 6)
('Royal Arena', 4)
('Taipei Arena', 10)
('Singapore Indoor Stadium', 5)
('Palais des Sports de Marseille', 3)
('Caja Magica', 3)
('All England Lawn Tennis and Croquet Club', 19)
('Stade Roland Garros', 18)
('Melbourne Park', 25)
('Qi zhong Forest Sports City Arena', 2)
('Wake Forest University Tennis Complex', 6)
('The Drive', 5)
(

In [18]:
#  Get venue details for a specific country (e.g., 'Chile')
cursor.execute("""
SELECT * FROM venues
WHERE country_name = 'Chile';
""")
print("\n3. Venues in Chile:")
for row in cursor.fetchall():
    print(row)


3. Venues in Chile:
('sr:venue:13830', 'Cancha 1', 'Santiago', 'Chile', 'CHL', 'America/Santiago', 'sr:complex:62419')
('sr:venue:13832', 'Cancha 3', 'Santiago', 'Chile', 'CHL', 'America/Santiago', 'sr:complex:62419')
('sr:venue:15858', 'Centre Court', 'Santiago', 'Chile', 'CHL', 'America/Santiago', 'sr:complex:62419')
('sr:venue:15860', 'Court 16', 'Santiago', 'Chile', 'CHL', 'America/Santiago', 'sr:complex:62419')
('sr:venue:15862', 'Court 15', 'Santiago', 'Chile', 'CHL', 'America/Santiago', 'sr:complex:62419')
('sr:venue:15864', 'Court 11', 'Santiago', 'Chile', 'CHL', 'America/Santiago', 'sr:complex:62419')
('sr:venue:1602', 'Cancha Central', 'Santiago', 'Chile', 'CHL', 'America/Santiago', 'sr:complex:62419')
('sr:venue:1603', 'Court 2', 'Santiago', 'Chile', 'CHL', 'America/Santiago', 'sr:complex:62419')
('sr:venue:1604', 'Cancha 4', 'Santiago', 'Chile', 'CHL', 'America/Santiago', 'sr:complex:62419')
('sr:venue:1605', 'Cancha 6', 'Santiago', 'Chile', 'CHL', 'America/Santiago', 'sr:

In [19]:
 # Identify all venues and their timezones
cursor.execute("""
SELECT venue_name, timezone FROM venues;
""")
print("\n4. Venues and Timezones:")
for row in cursor.fetchall():
    print(row)


4. Venues and Timezones:
('Estadio Monumental', 'America/Lima')
('Stadium', 'Asia/Shanghai')
('Court 2', 'Asia/Shanghai')
('Court 7', 'Australia/Sydney')
('Court 20', 'Australia/Melbourne')
('Court 22', 'Australia/Melbourne')
('Estadio Ivan Elias Moreno', 'America/Lima')
('Stade Numa-Daly Magenta', 'Pacific/Noumea')
('Noevir Stadium', 'Asia/Tokyo')
('Shanghai Stadium', 'Asia/Shanghai')
('Cancha 5', 'America/Mexico_City')
('Court 13', 'America/Los_Angeles')
('Center Court', 'Africa/Casablanca')
('Court 1', 'Africa/Casablanca')
('Court 2', 'Africa/Casablanca')
('Court 4', 'Africa/Casablanca')
('SZ Court', 'Europe/Berlin')
('Court 1', 'Europe/Lisbon')
('Center Court', 'Europe/Berlin')
('Show Court', 'Europe/Berlin')
('Court 1', 'Europe/Berlin')
('Court 2', 'Europe/Berlin')
('Court 9', 'Europe/Rome')
('Estadio Alejandro Villanueva', 'America/Lima')
('Court 11', 'Europe/Rome')
('Halle des Sports', 'Europe/Paris')
('Court 4', 'America/New_York')
('Arthur Ashe Stadium', 'America/New_York')
(

In [20]:
# Find complexes that have more than one venue
cursor.execute("""
SELECT c.complex_name, COUNT(v.venue_id) AS venue_count
FROM venues v
JOIN complexes c ON v.complex_id = c.complex_id
GROUP BY c.complex_name
HAVING COUNT(v.venue_id) > 1;
""")
print("\n5. Complexes with More Than One Venue:")
for row in cursor.fetchall():
    print(row)


5. Complexes with More Than One Venue:
('Kindarena', 4)
('Estadio de la Cartuja', 2)
('Estadio German Becker', 4)
('Palais des sports de Gerland', 7)
('Sibur Arena', 4)
('Palais Des Sport', 7)
('Palasport', 4)
('Salle Steredenn', 6)
('Porsche-Arena', 3)
('Rotterdam Ahoy', 3)
('Bluesun Tennis Center', 8)
('Club Sonoma, Estadio Gnp Seguros', 3)
('Accor Arena', 2)
('Roy Emerson Arena', 4)
('Complexo de Tenis do Jamor', 5)
('Club de Tenis Puente Romano', 9)
('National Tennis Center', 63)
('Canberra Tennis Centre', 7)
('BT Arena', 2)
('Valletta Cambiaso ASD', 9)
('Porsche Arena', 6)
('Royal Arena', 4)
('Taipei Arena', 10)
('Singapore Indoor Stadium', 5)
('Palais des Sports de Marseille', 3)
('Caja Magica', 3)
('All England Lawn Tennis and Croquet Club', 19)
('Stade Roland Garros', 18)
('Melbourne Park', 25)
('Qi zhong Forest Sports City Arena', 2)
('Wake Forest University Tennis Complex', 6)
('The Drive', 5)
('Atlantic Station', 4)
('International Tennis Hall of Fame', 7)
('Club Jacaranda 

In [21]:
# List venues grouped by country
cursor.execute("""
SELECT country_name, COUNT(*) AS venue_count
FROM venues
GROUP BY country_name;
""")
print("\n6. Venues Grouped by Country:")
for row in cursor.fetchall():
    print(row)


6. Venues Grouped by Country:
('Peru', 37)
('China', 207)
('Australia', 100)
('New Caledonia', 6)
('Japan', 101)
('Mexico', 120)
('USA', 501)
('Morocco', 10)
('Germany', 134)
('Portugal', 78)
('Italy', 251)
('France', 246)
('Argentina', 84)
('Romania', 47)
('Uzbekistan', 21)
('Republic of Korea', 35)
('Brazil', 101)
('Thailand', 19)
('Kazakhstan', 37)
('Chile', 65)
('Austria', 40)
('Sweden', 15)
('Russia', 22)
('Tunisia', 21)
('Chinese Taipei', 30)
('Czechia', 60)
('Luxembourg', 5)
('England', 112)
('Slovenia', 11)
('Turkiye', 52)
('Canada', 65)
('Croatia', 62)
('Finland', 13)
('Spain', 205)
('San Marino', 3)
('Hong Kong, China', 19)
('Bosnia & Herzegovina', 9)
('Poland', 37)
('Ecuador', 54)
('Qatar', 12)
('Switzerland', 49)
('India', 22)
('Singapore', 8)
('Slovakia', 20)
('Uruguay', 9)
('New Zealand', 14)
('United Arab Emirates', 17)
('Belgium', 6)
('Dominican Republic', 21)
('Netherlands', 17)
('Colombia', 45)
('Monaco', 9)
('Serbia', 21)
('Bulgaria', 21)
('Hungary', 23)
('Ukraine',

In [22]:
# Find all venues for a specific complex (e.g., 'Nacional')
cursor.execute("""
SELECT v.venue_name
FROM venues v
JOIN complexes c ON v.complex_id = c.complex_id
WHERE c.complex_name = 'Nacional';
""")
print("\n7. Venues in 'Nacional' Complex:")
for row in cursor.fetchall():
    print(row)


7. Venues in 'Nacional' Complex:
('Cancha Central',)


In [23]:
# COLLECT THE DOUBLES COMPETITOR RANKINGS DATA FROM THE API ENDPOINTS

In [24]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Melmir@123",
    database="david"
)
cursor = conn.cursor()

# Create competitors table
cursor.execute("""
CREATE TABLE IF NOT EXISTS competitors (
    competitor_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    country VARCHAR(100) NOT NULL,
    country_code CHAR(3) NOT NULL,
    abbreviation VARCHAR(10) NOT NULL
);
""")

# Create competitor_rankings table
cursor.execute("""
CREATE TABLE IF NOT EXISTS competitor_rankings (
    rank_id INT AUTO_INCREMENT PRIMARY KEY,
    rank_position INT NOT NULL,
    movement INT NOT NULL,
    points INT NOT NULL,
    competitions_played INT NOT NULL,
    competitor_id VARCHAR(50),
    FOREIGN KEY (competitor_id) REFERENCES competitors(competitor_id)
);
""")

print("✅ Tables created successfully.")
cursor.close()
conn.close()

✅ Tables created successfully.


In [25]:
import requests
import json

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json"
headers = {
    "accept": "application/json",
    "x-api-key": "1k3d2rnV453wSlaFM5SHAxf7PwoYTNj9CS6SqukZ"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    with open("rankings.json", "w") as f:
        json.dump(data, f, indent=2)
    print("✅ Rankings data saved to rankings.json")
else:
    print(f"❌ Failed to fetch data: {response.status_code}")
    print(response.text[:300])

✅ Rankings data saved to rankings.json


In [26]:
import json
import mysql.connector

# Load local JSON file
with open("rankings.json", "r") as f:
    data = json.load(f)

rankings = data.get("rankings", [])

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Melmir@123",
    database="david"
)
cursor = conn.cursor()

for item in rankings:
    comp = item.get("competitor", {})
    competitor_id = comp.get("id")

    if not competitor_id or "rank" not in item:
        continue

    # Insert into competitors
    cursor.execute("""
        INSERT IGNORE INTO competitors (
            competitor_id, name, country, country_code, abbreviation
        ) VALUES (%s, %s, %s, %s, %s)
    """, (
        competitor_id,
        comp.get("name", ""),
        comp.get("country", ""),
        comp.get("country_code", ""),
        comp.get("abbreviation", "")
    ))

    # Insert into rankings
    cursor.execute("""
        INSERT INTO competitor_rankings (
            rank_position, movement, points, competitions_played, competitor_id
        ) VALUES (%s, %s, %s, %s, %s)
    """, (
        item.get("rank"),
        item.get("movement"),
        item.get("points"),
        item.get("competitions_played"),
        competitor_id
    ))

conn.commit()
print("✅ Data inserted into MySQL successfully.")
cursor.close()
conn.close()

✅ Data inserted into MySQL successfully.


In [27]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",        
    password="Melmir@123",    
    database="david"     
)
cursor = conn.cursor()

# Create Competitors Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS competitors (
    competitor_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    country VARCHAR(100) NOT NULL,
    country_code CHAR(3) NOT NULL,
    abbreviation VARCHAR(10) NOT NULL
);
""")

# Create Competitor Rankings Table
cursor.execute("""
CREATE TABLE IF NOT EXISTS competitor_rankings (
    rank_id INT AUTO_INCREMENT PRIMARY KEY,
    rank_position INT NOT NULL,
    movement INT NOT NULL,
    points INT NOT NULL,
    competitions_played INT NOT NULL,
    competitor_id VARCHAR(50),
    FOREIGN KEY (competitor_id) REFERENCES competitors(competitor_id)
);
""")

conn.commit()
print("✅ Tables created for competitors and rankings.")

✅ Tables created for competitors and rankings.


In [28]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",        
    password="Melmir@123",    
    database="david" 
)
cursor = conn.cursor()

# ✅ List all tables in the connected database
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
print("✅ Tables in database:")
for (table_name,) in tables:
    print(f"- {table_name}")

✅ Tables in database:
- categories
- competitions
- competitor_rankings
- competitors
- complexes
- marks
- students
- venues


In [29]:
# Show all column names of competitor_rankings
cursor.execute("SHOW COLUMNS FROM competitor_rankings")
columns = cursor.fetchall()
print("Columns in competitor_rankings:")
for col in columns:
    print(col[0])

Columns in competitor_rankings:
rank_id
rank_val
movement
points
competitions_played
competitor_id
year


In [30]:
# Get all competitors with their rank and points
cursor.execute("""
SELECT c.name, r.rank_val, r.points
FROM competitor_rankings r
JOIN competitors c ON r.competitor_id = c.competitor_id;
""")
for row in cursor.fetchall():
    print(row)

('Pavic, Mate', 1, 8280)
('Arevalo-Gonzalez, Marcelo', 1, 8280)
('Heliovaara, Harri', 3, 8060)
('Patten, Henry', 3, 8060)
('Krawietz, Kevin', 5, 6515)
('Putz, Tim', 6, 6425)
('Granollers, Marcel', 7, 6135)
('Zeballos, Horacio', 8, 6135)
('Thompson, Jordan', 9, 5440)
('Glasspool, Lloyd', 10, 5140)
('Skupski, Neal', 11, 5050)
('Cash, Julian', 12, 4705)
('Mektic, Nikola', 13, 4670)
('Bolelli, Simone', 14, 4440)
('Vavassori, Andrea', 15, 4440)
('Purcell, Max', 16, 4410)
('Salisbury, Joe', 17, 3840)
('King, Evan', 18, 3770)
('Harrison, Christian', 19, 3730)
('Venus, Michael', 20, 3710)
('Roger-Vasselin, Edouard', 21, 3105)
('Molteni, Andres', 22, 2955)
('Reboul, Fabien', 23, 2905)
('Peers, John', 24, 2800)
('Nys, Hugo', 25, 2770)
('Doumbia, Sadio', 26, 2640)
('Goransson, Andre', 27, 2630)
('Lammons, Nathaniel', 28, 2620)
('Withrow, Jackson', 29, 2620)
('Arends, Sander', 30, 2610)
('Verbeek, Sem', 31, 2585)
('Gonzalez, Maximo', 32, 2550)
('Johnson, Luke', 33, 2480)
('Murray, Jamie', 34, 2460

In [31]:
# Find competitors ranked in the top 5
cursor.execute("""
SELECT c.name, r.rank_val, r.points
FROM competitor_rankings r
JOIN competitors c ON r.competitor_id = c.competitor_id
WHERE r.rank_val <= 5;
""")
for row in cursor.fetchall():
    print(row)

('Pavic, Mate', 1, 8280)
('Arevalo-Gonzalez, Marcelo', 1, 8280)
('Heliovaara, Harri', 3, 8060)
('Patten, Henry', 3, 8060)
('Krawietz, Kevin', 5, 6515)


In [32]:
# List competitors with no rank movement (stable rank)
cursor.execute("""
SELECT c.name, r.rank_val, r.points, r.movement
FROM competitor_rankings r
JOIN competitors c ON r.competitor_id = c.competitor_id
WHERE r.movement = 0;
""")
for row in cursor.fetchall():
    print(row)

('Pavic, Mate', 1, 8280, 0)
('Arevalo-Gonzalez, Marcelo', 1, 8280, 0)
('Heliovaara, Harri', 3, 8060, 0)
('Patten, Henry', 3, 8060, 0)
('Krawietz, Kevin', 5, 6515, 0)
('Putz, Tim', 6, 6425, 0)
('Granollers, Marcel', 7, 6135, 0)
('Zeballos, Horacio', 8, 6135, 0)
('Salisbury, Joe', 17, 3840, 0)
('Roger-Vasselin, Edouard', 21, 3105, 0)
('Peers, John', 24, 2800, 0)
('Nys, Hugo', 25, 2770, 0)
('Doumbia, Sadio', 26, 2640, 0)
('Goransson, Andre', 27, 2630, 0)
('Johnson, Luke', 33, 2480, 0)
('Murray, Jamie', 34, 2460, 0)
('Bhambri, Yuki', 35, 2350, 0)
('Galloway, Robert', 36, 2330, 0)
('Ram, Rajeev', 37, 2215, 0)
('Zielinski, Jan', 41, 2085, 0)
('Erler, Alexander', 43, 2062, 0)
('Romboli, Fernando', 51, 1890, 0)
('Andreozzi, Guido', 52, 1830, 0)
('Melo, Marcelo', 57, 1660, 0)
('Rikl, Patrik', 64, 1435, 0)
('Nouza, Petr', 64, 1435, 0)
('Vliegen, Joran', 66, 1400, 0)
('Tracy, James', 67, 1337, 0)
('Bollipalli, Rithvik Choudary', 80, 1154, 0)
('Matuszewski, Piotr', 81, 1132, 0)
('Pel, David', 82, 

In [33]:
# Get the total points of competitors from a specific country (e.g., Croatia)
country = "Croatia"
cursor.execute("""
SELECT SUM(r.points)
FROM competitor_rankings r
JOIN competitors c ON r.competitor_id = c.competitor_id
WHERE c.country = %s;
""", (country,))
print(f"Total points for {country}: {cursor.fetchone()[0]}")

Total points for Croatia: 15483


In [34]:
# Count the number of competitors per country
cursor.execute("""
SELECT c.country, COUNT(*) AS num_competitors
FROM competitors c
GROUP BY c.country
ORDER BY num_competitors DESC;
""")
for row in cursor.fetchall():
    print(row)

('USA', 46)
('France', 36)
('Great Britain', 29)
('Australia', 27)
('Italy', 25)
('Argentina', 24)
('India', 22)
('Japan', 20)
('Germany', 19)
('Spain', 18)
('Czechia', 17)
('Brazil', 16)
('Netherlands', 15)
('Neutral', 12)
('Austria', 10)
('Canada', 10)
('Switzerland', 8)
('China', 8)
('Croatia', 7)
('Ukraine', 7)
('Portugal', 6)
('Republic of Korea', 6)
('Poland', 6)
('Serbia', 5)
('Colombia', 5)
('Slovakia', 5)
('Kazakhstan', 5)
('Sweden', 4)
('Chile', 4)
('Belgium', 4)
('Finland', 4)
('New Zealand', 4)
('Romania', 4)
('Mexico', 4)
('Bulgaria', 4)
('Uruguay', 4)
('Chinese Taipei', 4)
('Bolivia', 4)
('Greece', 3)
('Peru', 3)
('Ecuador', 3)
('Tunisia', 3)
('Venezuela', 3)
('Turkiye', 2)
('Denmark', 2)
('Israel', 2)
('Zimbabwe', 2)
('Monaco', 2)
('South Africa', 2)
('Jamaica', 1)
('Philippines', 1)
('Lebanon', 1)
('Antigua and Barbuda', 1)
('Uzbekistan', 1)
('Estonia', 1)
('Georgia', 1)
('Morocco', 1)
('Jordan', 1)
('Bosnia & Herzegovina', 1)
('El Salvador', 1)
('Thailand', 1)
('Northe

In [35]:
# Find competitors with the highest points in the current week
cursor.execute("""
SELECT c.name, r.rank_val, r.points
FROM competitor_rankings r
JOIN competitors c ON r.competitor_id = c.competitor_id
WHERE r.points = (SELECT MAX(points) FROM competitor_rankings);
""")
for row in cursor.fetchall():
    print(row)

('Pavic, Mate', 1, 8280)
('Arevalo-Gonzalez, Marcelo', 1, 8280)
